<a href="https://colab.research.google.com/github/aist2000/ML-public/blob/master/el2/TCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TCN method

In [3]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Activation, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping

def create_dataset(dataset, window_size=1, stride=1):
    """
    Creates a dataset of sliding windows.

    Args:
      dataset: The input dataset as a NumPy array.
      window_size: The size of the sliding window.
      stride: The stride for the sliding window.

    Returns:
      A NumPy array of windows.
    """
    X = list()
    for i in range(window_size, len(dataset) - window_size + 1, stride):
        X.append(dataset[i-window_size:i, 1:])  # Exclude time column
    return np.array(X)

def train_model(X_train):
    """
    Trains a TCN autoencoder model for anomaly detection.

    Args:
      X_train: The training input data.

    Returns:
      A trained TCN autoencoder model.
    """
    num_features = X_train.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(window_size, num_features), padding='same', dilation_rate=1))
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', dilation_rate=2))
    model.add(Conv1D(filters=16, kernel_size=3, activation='relu', padding='same', dilation_rate=4))
    # Add more Conv1D layers with increasing dilation rates as needed

    # Decoder (upsampling and reconstruction)
    model.add(Conv1D(filters=16, kernel_size=3, activation='relu', padding='same', dilation_rate=4))
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', dilation_rate=2))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', dilation_rate=1))
    model.add(TimeDistributed(Dense(num_features)))  # Reconstruct the input

    model.compile(optimizer='adam', loss='mse')
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    model.fit(X_train, X_train, epochs=100, batch_size=32, verbose=1, validation_split=0.2, callbacks=[early_stopping])
    return model

def predict_anomalies(model, X_test, threshold=0.1, stride=1):
    """
    Predicts anomaly windows and observations based on the trained model and a threshold.

    Args:
      model: The trained TCN autoencoder model.
      X_test: The test input data.
      threshold: The threshold for anomaly detection.
      stride: The stride used for creating the sliding windows.

    Returns:
      A tuple containing:
        - anomaly_windows: A list of indices for windows containing anomalies.
        - anomaly_observations: A list of indices for observations considered anomalous.
    """
    predictions = model.predict(X_test)
    mse = np.mean(np.square(predictions - X_test), axis=2)  # Calculate MSE for each time step
    threshold = np.percentile(mse, 95)  # Example threshold (95th percentile)

    anomaly_windows = np.where(np.any(mse > threshold, axis=1))[0]  # Identify anomaly windows
    anomaly_observations = list()
    for window_index in anomaly_windows:
        window = mse[window_index]
        anomaly_indices = np.where(window > threshold)[0]  # Find anomalies within the window
        # Adjust indices to correspond to original dataset
        anomaly_observations.extend(window_index * stride + anomaly_indices)

    return anomaly_windows, anomaly_observations



In [4]:
#import numpy as np
data = np.array([
   [100,0, 0], [101,0, 0],[102,0, 1],[103,0, 1],[104,0, 1],[105,1, 1],[106,0, 1],[107,0, 1],[108,0, 1],[109,1, 1],[110,0, 1]
              ,[111,0, 0], [112,0, 0],[113,0, 1],[114,0, 1],[115,0, 1],[116,1, 1],[117,0, 1],[118,0, 1],[119,0, 1],[120,1, 1]
              ,[121,0, 0], [122,0, 0],[123,0, 1],[124,0, 1],[125,0, 1],[126,1, 1],[127,0, 1],[128,0, 1],[129,0, 1],[130,1, 1]
  ])
url="https://raw.githubusercontent.com/aist2000/ML-public/master/dataset.csv"
#url="dataset.csv"
df = pd.read_csv(url, skiprows=2, header=None)
display(df.head())
display(df.describe())

In [5]:
# Example usage:
# Assuming your data is in a NumPy array called 'data' with shape (num_samples, 31)
# where the first column is time and the rest are the 30 features


# 1. Data Preprocessing
scaler = StandardScaler()
data[:, 1:] = scaler.fit_transform(data[:, 1:])  # Standardize features
window_size = 4 #10000  # Set window size to 10,000
stride = 2      # 1000  # Adjust stride as needed
X = create_dataset(data, window_size, stride)

# 2. Train-Test Split (adjust split ratio as needed)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]

# 3. Model Training
model = train_model(X_train)



Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.7107 - val_loss: 0.4943
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step - loss: 0.7033 - val_loss: 0.4905
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.6960 - val_loss: 0.4867
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.6884 - val_loss: 0.4826
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.6803 - val_loss: 0.4782
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.6712 - val_loss: 0.4734
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.6612 - val_loss: 0.4685
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.6502 - val_loss: 0.4634
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - loss: 0.6382 - val_loss: 0.4583
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.6254 - val_loss: 0.4532
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.6116 - val_loss: 0.4483
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.5967 - val_loss: 0.4436
E

In [10]:
print(X_test)

[[[ 0  0]
  [ 0  0]
  [ 2  0]
  [ 0 -2]]

 [[ 2  0]
  [ 0 -2]
  [ 0 -2]
  [ 0  0]]

 [[ 0 -2]
  [ 0  0]
  [ 0  0]
  [ 0  0]]]


In [6]:
# 4. Anomaly Prediction
threshold = 0.1  # Adjust threshold as needed

anomaly_windows, anomaly_observations = predict_anomalies(model, X_test, threshold, stride)

# Print the results
print("Anomaly Windows:", anomaly_windows)
print("Anomaly Observations:", anomaly_observations)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Anomaly Windows: [0]
Anomaly Observations: [3]
